In [ ]:
import os
os.environ['ANTHROPIC_API_KEY'] = ""
create_magic()

In [1]:
# import aimagic
# from aimagic import *
import transformers
from transformers import PreTrainedModel, BertConfig, AutoTokenizer, AutoModel
from src.bert_layers.configuration_bert import FlexBertConfig
from src.bert_layers.model import *
from src.flex_bert import create_flex_bert_classification
from main import init_from_checkpoint
import omegaconf
from omegaconf import OmegaConf

import torch
import torch.nn as nn

/home/public/span/miniconda3/envs/bert24/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/public/span/miniconda3/envs/bert24/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/home/public/span/miniconda3/envs/bert24/lib/python3.11/site-packages/flash_attn/ops/triton/layer_norm.py:958: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/home/public/span/miniconda3/envs/bert24/lib/python3.11/site-packages/flash_attn/ops/triton/layer_norm.py:1017: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., 

In [2]:
config = FlexBertConfig()
config.num_labels = 1
model = FlexBertForSequenceClassification(config)

In [12]:
omega_conf_cfg = OmegaConf.load("yamls/test/main_bert24_base.yaml")
init_from_checkpoint(omega_conf_cfg, model)

Initializing model from checkpoint experiment_001


TypeError: expected str, bytes or os.PathLike object, not DictConfig

In [8]:
tokenizer = AutoTokenizer.from_pretrained("bclavie/olmo_bert_template")

In [9]:
model = model.to("cuda:1")

In [10]:
test_text = ".."
tokenized =  tokenizer(test_text, return_tensors="pt")
tokenized['input_ids'].to(torch.bfloat16)

tensor([[50176.,   536., 50176.]], dtype=torch.bfloat16)

In [13]:
tokenized

{'input_ids': tensor([[50281,   537, 50282]]), 'token_type_ids': tensor([[0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1]])}

In [22]:
current_input = tokenized

for layer in model.model.modules():
    print(layer)
    print(current_input)
    current_input = layer(current_input)
    

FlexBertForSequenceClassification(
  (bert): FlexBertModel(
    (embeddings): FlexBertAbsoluteEmbeddings(
      (tok_embeddings): Embedding(30528, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (norm): RMSNorm()
      (drop): Identity()
    )
    (encoder): FlexBertUnpadEncoder(
      (layers): ModuleList(
        (0-11): 12 x FlexBertUnpadPreNormLayer(
          (attn_norm): RMSNorm()
          (attn): FlexBertUnpadAttention(
            num_attention_heads=12, attn_head_size=64, sliding_window=False, use_fa2=True, deterministic_fa2=False
            (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
            (Wo): Linear(in_features=768, out_features=768, bias=False)
            (out_drop): Identity()
          )
          (mlp_norm): RMSNorm()
          (mlp): FlexBertMLP(
            (Wi): Linear(in_features=768, out_features=3072, bias=False)
            (act): GELU(approximate='none')
            (drop): Identity()
            (Wo): Line

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
